Simple implementation of model-free REINFORCE algorithm, which is a policy-gradient based method.
The code procedure is:

1. Initialize policy parameters θ
2. For each episode do:
    2.1. Generate an episode trajectory τ = {(s_1, a_1, r_1), ..., (s_T, a_T, r_T)} by following the policy π_θ
    2.2. For each time step t in the episode do:
    
        2.2.1. Calculate the return G_t = r_{t+1} + γr_{t+2} + ... + γ^(T-t)r_T

        2.2.2. Calculate the gradient of the policy with respect to the parameters θ: ∇_θ log(π_θ(a_t|s_t)) * G_t
        
        2.2.3. Update the policy parameters: θ = θ + α * ∇_θ log(π_θ(a_t|s_t)) * G_t
3. Repeat steps 2-3 until the policy converges or a stopping criterion is met

where 

- θ are the policy parameters.
- π_θ(a_t|s_t) is the probability of taking action a_t in state s_t under policy π parameterized by θ.
- G_t is the return from time step t onwards.
- γ is the discount factor, which balances the importance of immediate and future rewards.
- α is the learning rate, which controls the step size of the gradient ascent update.
- T is the number of time steps in the episode.

In [ ]:
# need a virtual display for rendering in docker
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
import jax
import jax.numpy as jnp
import gymnasium as gym
import optax
from functools import partial 

env = gym.make('CartPole-v1', render_mode="rgb_array").env


# Define the policy
def policy(state, params):
    W, b = params
    logits = jnp.dot(W, state) + b
    probs = jax.nn.softmax(logits)
    action = jax.random.categorical(jax.random.PRNGKey(0), logits)
    return action, probs

# Define the loss function


def loss(params, rollout):
    A, B, W, b = params
    states, actions, rewards, next_states = rollout

    # Compute log likelihood of actions taken by the policy
    _, action_probs = jax.vmap(policy, in_axes=(0, None))(states, (W, b))
    log_probs = jnp.log(
        jnp.sum(action_probs * jax.nn.one_hot(actions, 2), axis=-1))

    # Compute the loss
    loss = -jnp.mean(log_probs * rewards)

    return loss


# Define the optimizer
optimizer = optax.adam(learning_rate=1e-3)

# Initialize parameters
key = jax.random.PRNGKey(0)
A = jax.random.normal(key, shape=(4, 4))
B = jax.random.normal(key, shape=(4,))
W = jax.random.normal(key, shape=(2, 4))
b = jax.random.normal(key, shape=(2,))

params = (A, B, W, b)

# Define the rollout function


def rollout(params, rng_key, env):
    states = []
    actions = []
    rewards = []
    next_states = []

    A, B, W, b = params
    state, _ = env.reset()
    done = False

    while not done:
        action, probs = policy(state, (W,b))
        print(probs)
        print(action)
        next_state, reward, done, _, _ = env.step(int(action))
        states.append(state)
        actions.append(action)
        rewards.append(reward)
        next_states.append(next_state)
        state = next_state

    return (jnp.stack(states), jnp.array(actions), jnp.array(rewards), jnp.stack(next_states))



In [ ]:
# add some test
rollout_fn = jax.tree_util.Partial(rollout, params)
rollout_fn(key, env)

In [ ]:

# Train the model
states_opt = optimizer.init(params)
for i in range(10000):
    key, subkey = jax.random.split(key)
    
    # this seems not working as [env] is not treated as jnp object
    #rollout_fn = jax.jit(partial(rollout, params))
    #rollout_batch = jax.vmap(rollout_fn)(
    #    jax.random.split(subkey, 10), [env] * 10)
    rollout_fn = partial(rollout, params)
    rollout_batch = rollout_fn(key, env)

    grads = jax.grad(loss)(params, rollout_batch)
    updates, states_opt = optimizer.update(grads, states_opt, params)
    params = optax.apply_updates(params, updates)

    if i % 100 == 0:
        print(f'Iteration {i}, loss={loss(params, rollout_batch):.4f}')


In [ ]:
params
len(params)
params[2:]


In [ ]:
from IPython import display as ipythondisplay
import matplotlib.pyplot as plt 

# Test the model
state, _ = env.reset(seed=10)
done = False
while not done:
    action, _ = policy(state, params[2:])
    state, _, done, _, _ = env.step(int(action))
    screen = env.render()
    plt.imshow(screen)
    ipythondisplay.clear_output(wait=True)
    ipythondisplay.display(plt.gcf()) 

ipythondisplay.clear_output(wait=True)
env.close()
